In [1]:
%run TreePrinter.ipynb

In [2]:
import findspark
findspark.init('/usr/local/spark')

In [3]:
import pyspark,pyodbc

In [4]:
from pyspark import SparkContext, SparkConf, SQLContext
import _mssql
import pandas as pd

In [5]:
master="local"
conf = SparkConf() \
    .setAppName("pysparkToAzure") \
    .setMaster(master) \
    .set("spark.driver.extraClassPath","/home/mosaico/Documents/mosaico/sqljdbc_8.2/mssql-jdbc-8.2.0.jre11.jar")\
    .set("spark.executor.extraClassPath","/home/mosaico/Documents/mosaico/sqljdbc_8.2/mssql-jdbc-8.2.0.jre11.jar")
sc=pyspark.SparkContext(conf=conf)

sc.stop()

In [6]:
sc.appName

'pysparkToAzure'

In [7]:
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [8]:
database = "Test"
ser="mosaico.database.windows.net,1433"
table = "dbo.DOCENTI"
user = "PanDario"
password = "Mosaico2020"


In [11]:
conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={ser};DATABASE={database};UID={user};PWD={password}')

In [12]:
def listTables(res:pd.core.frame.DataFrame):
    obj=res['TABLE_NAME']
    list=[]
    for x in obj:
        list.append(x)
    return list
def listCols(res:pd.core.frame.DataFrame):
    obj=res.columns
    list=[]
    for c in obj:
        list.append(c)
    return list

QUERY DI TEST

In [13]:
query="SELECT * FROM  INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'"
res=pd.read_sql(query, conn)
tablesName=listTables(res)
tables=[]
root=Node("Test",'DB')
for t in tablesName:
    tables.append(Node(t,'table'))
root.add_child(tables)
for t in tables:
    query="SELECT * FROM  {0} ".format(t.data)
    res=pd.read_sql(query, conn)
    leafs=listCols(res)
    for l in leafs:
        n=Node(l,'column')
        t.add_child(n)

pprint_tree(root)

`- Test
   |- STUDENTI
   |  |- NOME
   |  |- COGNOME
   |  |- CF
   |  `- IBAN
   `- PROFESSORI
      |- NOME
      `- COGNOME


### TEST DATAFRAME

In [14]:
sparkDF =  spark.createDataFrame(res)

### Cali for ODRL

In [1]:
import pycali

In [2]:
from pycali.vocabulary import ODRLVocabulary
odrl = ODRLVocabulary()

#odrl.actions

### AD HOC USER
CREATE USER Tester WITH PASSWORD = 'SuperSecure.1'; <br>
SELECT * FROM sys.database_principals WHERE authentication_type_desc='DATABASE'

### SQL QUERY
sql query has this structure
<pre>
SELECT something FROM some table
   WHERE some conditions are satisfied

UPDATE some table SET something
   WHERE some conditions are satisfied
<code>

### ODRL POLICY TEST

In [15]:
import json,re
data={}
with open("policy.json") as policy:
    data=json.load(policy)
print(json.dumps(data, indent=2))

{
  "@context": "http://donnowhatisit.org",
  "@type": "Agreement",
  "uid": "mosaico.database.windows.net/Test:policy:1",
  "permission": [
    {
      "target": "mosaico.database.windows.net/Test:STUDENTI",
      "assignee": "mosaico.database.windows.net/Test:user:Tester",
      "action": "read"
    },
    {
      "target": "mosaico.database.windows.net/Test:DOCENTI:NOME",
      "assignee": "mosaico.database.windows.net/Test:user:Tester",
      "action": "read"
    }
  ]
}


In [16]:
MAINSERVER="mosaico.database.windows.net/Test"

def jsonPermissionSearch(data,assignee,targetColumn,targetTable):
    for e in data['permission']:
        #if rule that define access to the whole table
        if e['assignee']=="{0}:user:{1}".format(MAINSERVER,assignee) and e['target']=="{0}:{1}".format(MAINSERVER,targetTable):
            #print("rule for the whole table found")
            return e
        #rule specific to table:column
        elif e['assignee']=="{0}:user:{1}".format(MAINSERVER,assignee) and e['target']=="{0}:{1}:{2}".format(MAINSERVER,targetTable,targetColumn) and targetColumn!='*':
            #print("rule for the column found")
            return e
        else:
            pass
    #print('no rule found')
    return None

In [17]:
query="SELECT * FROM STUDENTI ;SELECT NOME,COGNOME FROM DOCENTI;"
query=query.split(';')
#multiple query split-check for select, not for updates
completeQueryList=[]
for q in query:
    if len(q.strip())>0:
        res=re.split('SELECT|FROM',q)
        res.remove('')
        res=[r.strip() for r in res]
        #res[0] -> select args
        #res[1] -> from args
        for column in res[0].split(','):
            policy=jsonPermissionSearch(data,'Tester',column,res[1])
            if policy and policy['action']=='read':
                completeQueryList.append('SELECT {0} FROM {1}'.format(column,res[1]))
print(';'.join(completeQueryList))


SELECT * FROM STUDENTI;SELECT NOME FROM DOCENTI


#### TODO
Aggregare query appartenenti alla stessa tabella per evitare query eccessive.  
Se devo selezionare ad esempio nella tabella ALPHA le tuple con NOME==a , NOME==b , NOME==c  deve risultare in :  
    SELECT * FROM ALPHA WHERE NAME in ('a', 'b' , 'c')

## RDFLIB time

In [3]:
#!pip3 install rdflib-jsonld
from rdflib import Graph, plugin
jsonData='''
[{
  "@type": "Agreement",
  "uid": "mosaico.database.windows.net/Test:policy:1",
  "permission": [{
      "target": "mosaico.database.windows.net/Test:STUDENTI",
      "assignee": "mosaico.database.windows.net/Test:user:Tester",
      "action": "read"
  }]
  },
  {
  "@type": "Agreement",
  "uid": "mosaico.database.windows.net/Test:policy:1",
  "permission": [
    {
      "target": "mosaico.database.windows.net/Test:DOCENTI:NOME",
      "assignee": "mosaico.database.windows.net/Test:user:Tester",
      "action": "read"
    }
  ]
}]
'''

In [4]:
import rdflib
g=Graph().parse(data=jsonData,format='json-ld')
g.serialize(format='nt')

b'_:N43afefaddae44a1681f62bbb642f6bd6 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <file:///home/mosaico/Documents/mosaico/Agreement> .\n_:Ncf6291d1b875416c97cfca231839bbf8 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <file:///home/mosaico/Documents/mosaico/Agreement> .\n\n'

In [22]:
import pprint
okString='''{
    "@id": "http://example.org/about",
    "http://purl.org/dc/terms/title": [
        {
            "@type": "literal",
            "@language": "en",
            "@value": "Someone's Homepage"
        }
    ],
    "@authorization":"read"
}'''

print(isinstance(okString,dict))
g=Graph().parse(data=okString,format='json-ld')
list(g)

False


[(rdflib.term.URIRef('http://example.org/about'),
  rdflib.term.URIRef('http://purl.org/dc/terms/title'),
  rdflib.term.Literal("Someone's Homepage", lang='en'))]

## IRI Parsing